# 第二章 文档加载

 - [一、PDF文档](#一、PDF文档)
     - [1.1 安装相关包 ](#1.1-安装相关包-)
     - [1.2 加载PDF文档](#1.2-加载PDF文档)
     - [1.3 探索加载的数据](#1.3-探索加载的数据)
 - [二、网页文档](#二、网页文档)
     - [2.1 加载网页文档](#2.1-加载网页文档)
     - [2.2 探索加载的数据](#2.2-探索加载的数据)
 - [三、Notion文档](#三、Notion文档)
     - [3.1 加载Notion Markdown文档](#3.1-加载Notion-Markdown文档)
     - [3.2 探索加载的数据](#3.2-探索加载的数据)


同前序课程一样，我们同样需要先配置环境以访问 OpenAI 提供的 API，配置方法如下，详细介绍请参见其他课程

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 

openai.api_key  = os.environ['OPENAI_API_KEY']
openai.base_url = os.environ['OPENAI_API_BASE']

## 一、PDF文档

首先，我们来加载一个[PDF文档](https://see.stanford.edu/materials/aimlcs229/transcripts/MachineLearning-Lecture01.pdf)。该文档为吴恩达教授的2009年机器学习课程的字幕文件。因为这些字幕为自动生成，所以词句直接可能不太连贯和通畅。

### 1.1 安装相关包 

### 1.2 加载PDF文档

In [2]:
from langchain_community.document_loaders.pdf import PyPDFLoader

# 创建一个 PyPDFLoader Class 实例，输入为待加载的pdf文档路径
loader = PyPDFLoader("docs/cs229_lectures/MachineLearning-Lecture01.pdf")

# 调用 PyPDFLoader Class 的函数 load对pdf文件进行加载
pages = loader.load()

### 1.3 探索加载的数据

文档加载后储存在`pages`变量中:
- `page`的变量类型为`List`
- 打印 `pages` 的长度可以看到pdf一共包含多少页

In [3]:
print(type(pages))

<class 'list'>


In [4]:
print(len(pages))

22


`page`中的每一元素为一个文档，变量类型为`langchain.schema.document.Document`, 文档变量类型包含两个属性
- `page_content` 包含该文档的内容。
- `meta_data` 为文档相关的描述性数据。

In [5]:
page = pages[0]

In [6]:
print(type(page))

<class 'langchain_core.documents.base.Document'>


In [7]:
print(page.page_content[0:500])

MachineLearning-Lecture01  
Instructor (Andrew Ng):  Okay. Good morning. Welcome to CS229, the machine 
learning class. So what I wanna do today is ju st spend a little time going over the logistics 
of the class, and then we'll start to  talk a bit about machine learning.  
By way of introduction, my name's  Andrew Ng and I'll be instru ctor for this class. And so 
I personally work in machine learning, and I' ve worked on it for about 15 years now, and 
I actually think that machine learning i


In [8]:
print(page.metadata)

{'source': 'docs/cs229_lectures/MachineLearning-Lecture01.pdf', 'page': 0}


## 二、网页文档

本部分，对于给定网页文档链接(URLs)，我们学习如何对其进行加载。这里我们对Github上的网页文档进行加载，该文档格式为markdown。

### 2.1 加载网页文档

In [9]:
from langchain.document_loaders import WebBaseLoader


# 创建一个 WebBaseLoader Class 实例
url = "https://github.com/datawhalechina/llm-cookbook/blob/main/README.md"
header = {'User-Agent': 'python-requests/2.27.1', 
          'Accept-Encoding': 'gzip, deflate, br', 
          'Accept': '*/*',
          'Connection': 'keep-alive'}
loader = WebBaseLoader(web_path=url,header_template=header)

# 调用 WebBaseLoader Class 的函数 load对文件进行加载
docs = loader.load()

### 2.2 探索加载的数据

文档加载后储存在`docs`变量中:
- `docs`的变量类型为`List`
- 打印 `docs` 的长度可以看到一共包含多少页

In [10]:
print(type(docs))

<class 'list'>


In [11]:
print(len(docs))

1


`docs`中的每一元素为一个文档，变量类型为`langchain.schema.document.Document`, 文档变量类型包含两个属性
- `page_content` 包含该文档的内容。
- `meta_data` 为文档相关的描述性数据。

In [12]:
doc = docs[0]

In [13]:
print(type(doc))

<class 'langchain_core.documents.base.Document'>


In [14]:
print(doc.page_content)























































































llm-cookbook/README.md at main · datawhalechina/llm-cookbook · GitHub













































Skip to content












Navigation Menu

Toggle navigation










          Sign in
        


 













        Product
        












Actions
        Automate any workflow
      







Packages
        Host and manage packages
      







Security
        Find and fix vulnerabilities
      







Codespaces
        Instant dev environments
      







GitHub Copilot
        Write better code with AI
      







Code review
        Manage code changes
      







Issues
        Plan and track work
      







Discussions
        Collaborate outside of code
      




Explore



      All features

    



      Documentation

    





      GitHub Skills

    





      Blog

    









        Solutions
        





For



      Enterprise

    



  

langchain现在的版本会自动提取网页中的内容，但上面的文档内容仍包含许多冗余的信息。我们需要进行对这种数据进行进一步处理(Post Processing)。

In [15]:
start_index = doc.page_content.find("面向开发者的大模型手册")
end_index = doc.page_content.find("国际许可协议进行许可。")
print(doc.page_content[start_index:end_index])

面向开发者的大模型手册 - LLM Cookbook
项目简介
本项目是一个面向开发者的大模型手册，针对国内开发者的实际需求，主打 LLM 全方位入门实践。本项目基于吴恩达老师大模型系列课程内容，对原课程内容进行筛选、翻译、复现和调优，覆盖从 Prompt Engineering 到 RAG 开发、模型微调的全部流程，用最适合国内学习者的方式，指导国内开发者如何学习、入门 LLM 相关项目。
针对不同内容的特点，我们对共计 11 门吴恩达老师的大模型课程进行了翻译复现，并结合国内学习者的实际情况，对不同课程进行了分级和排序，初学者可以先系统学习我们的必修类课程，掌握入门 LLM 所有方向都需要掌握的基础技能和概念，再选择性地学习我们的选修类课程，在自己感兴趣的方向上不断探索和学习。
如果有你非常喜欢但我们还没有进行复现的吴恩达老师大模型课程，我们欢迎每一位开发者参考我们已有课程的格式和写法来对课程进行复现并提交 PR，在 PR 审核通过后，我们会根据课程内容将课程进行分级合并。欢迎每一位开发者的贡献！
在线阅读地址：面向开发者的 LLM 入门课程-在线阅读
PDF下载地址：面向开发者的 LLM 入门教程-PDF
英文原版地址：吴恩达关于大模型的系列课程
项目意义
LLM 正在逐步改变人们的生活，而对于开发者，如何基于 LLM 提供的 API 快速、便捷地开发一些具备更强能力、集成LLM 的应用，来便捷地实现一些更新颖、更实用的能力，是一个急需学习的重要能力。
由吴恩达老师与 OpenAI 合作推出的大模型系列教程，从大模型时代开发者的基础技能出发，深入浅出地介绍了如何基于大模型 API、LangChain 架构快速开发结合大模型强大能力的应用。其中，《Prompt Engineering for Developers》教程面向入门 LLM 的开发者，深入浅出地介绍了对于开发者，如何构造 Prompt 并基于 OpenAI 提供的 API 实现包括总结、推断、转换等多种常用功能，是入门 LLM 开发的经典教程；《Building Systems with the ChatGPT API》教程面向想要基于 LLM 开发应用程序的开发者，简洁有效而又系统全面地介绍了如何基于 ChatGPT API 打造完整的对话系统；《LangChain for LLM Applicat

## 三、Notion文档

- 点击[Notion示例文档](https://yolospace.notion.site/Blendle-s-Employee-Handbook-e31bff7da17346ee99f531087d8b133f)右上方复制按钮(Duplicate)，复制文档到你的Notion空间
- 点击右上方`⋯` 按钮，选择导出为Mardown&CSV。导出的文件将为zip文件夹
- 解压并保存mardown文档到本地路径`docs/Notion_DB/`

### 3.1 加载Notion Markdown文档

In [16]:
from langchain.document_loaders import NotionDirectoryLoader
loader = NotionDirectoryLoader("docs/Notion_DB")
docs = loader.load()

### 3.2 探索加载的数据

文档加载后储存在`docs`变量中:
- `docs`的变量类型为`List`
- 打印 `docs` 的长度可以看到一共包含多少页

In [17]:
print(type(docs))

<class 'list'>


In [18]:
print(len(docs))

51


`docs`中的每一元素为一个文档，变量类型为`langchain.schema.document.Document`, 文档变量类型包含两个属性
- `page_content` 包含该文档的内容。
- `meta_data` 为文档相关的描述性数据。

In [19]:
doc = docs[0]

In [20]:
print(type(doc))

<class 'langchain_core.documents.base.Document'>


In [21]:
print(doc.page_content[0:500])

# Firing

Saying goodbye to Blendle (from a colleague) and to a colleague (from Blendle) is a very normal and natural thing. When done right, it can even be a beautiful thing. 

We advise you to read the backdrop below first, but feel free to jump in right away with the 'Here's what you can do'-section :). General note: you do not have to do this alone, so please ask for advice and help!

- **Backdrop**
    
    Saying goodbye to Blendle (from a colleague) and to a colleague (from Blendle) is a 
